In [83]:
import os
import h5py
import numpy as np
import pandas as pd
from data_helpers import grouper
from tempConv import tempConvDecoder
from sklearn.metrics import confusion_matrix

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
## get and format data
# lfp power bands
neural_data_file = h5py.File('datasets/GRat31_636061_all_sorted_spikes.hdf5', 'r')
neural_data = np.asarray(neural_data_file['sorted_spikes']) # iterate through powerbands
neural_data.shape

(80690, 34)

In [80]:
head_signals_h5 = h5py.File('datasets/GRat31_636061_all_head_data.hdf5', 'r')
idx_start, idx_stop = [3,6]

# dx
head_signal = np.asarray(
    [np.asarray(head_signals_h5[key]) for key in head_signals_h5.keys()][0:9]
).T[:,idx_start:idx_stop]
print('head_signals shape: ', head_signal.shape)

# head_signals_keys = list(head_signals_h5.keys())[0:9][idx_start:idx_stop]
# head_signals_int = ['yaw_abs', 'roll_abs', 'pitch_abs']
# print('head_signals_keys intuitive: ', head_signals_int)

head_signals shape:  (80690, 3)


In [92]:
y = head_signal[:,0]
y[:100]

array([  3.02586149e-02,  -1.52645508e-02,   3.97888278e-03,
         6.13548277e-03,   4.27395620e-03,   6.47417621e-04,
         1.47317093e-02,   5.74976396e-03,  -3.29849665e-03,
         6.23799305e-03,  -2.22612779e-02,  -9.07869402e-03,
         6.65859188e-03,  -4.19948065e-03,  -9.34697509e-04,
        -6.85261915e-05,   1.47826750e-02,   5.51965667e-03,
         4.91836678e-03,   1.20928397e-02,  -2.17919344e-02,
        -2.16071886e-02,   2.09097435e-03,  -8.26242276e-03,
        -1.76371113e-02,   3.51037186e-04,   4.98269628e-02,
         6.46501294e-02,   6.02881088e-04,  -4.38898143e-02,
        -4.16498827e-02,  -3.36884089e-02,  -1.44245378e-02,
         2.99746635e-02,   1.69246031e-02,   1.01747322e-02,
         1.25945501e-01,   1.28004692e-01,   1.13247756e-01,
         9.84169144e-02,  -5.37452621e-02,  -4.25896564e-02,
        -1.48623268e-03,  -1.70661356e-02,   8.52043356e-03,
        -4.84822274e-03,   1.20149614e-03,   1.23278168e-03,
        -2.33169011e-03,

In [6]:
# make head signals binary
head_signal = np.sign(head_signal)
head_signal[head_signal<0] = 0
print('head_signals shape: ', head_signal.shape)

head_signals shape:  (80690,)


In [18]:
# make head signals thresholded binary for movement or no movement
head_signal[head_signal>0.1] = 1
head_signal[head_signal<-0.1] = 1
head_signal[head_signal<0.1] = 0
print('head_signals shape: ', head_signal.shape)

head_signals shape:  (80690,)


In [79]:
# y[y>0.1] = 1
y[y<0.1] = 0
print('head_signals shape: ', y.shape)
hist = np.histogram(y)
1-hist[0][-1]/(hist[0][0]+hist[0][-1])

head_signals shape:  (80690,)


0.71247986119717432

In [75]:
stats = []

In [93]:
TCD = tempConvDecoder(neural_data,y,['yaw_abs'],window=60, offset=30, nb_filter=10, eps=15, regressor=False, lr=0.000001)
TCD.fit()
R2s,rs = TCD.determine_fit()
stats.append([R2s, rs])

X_train and y_train shape: (3195, 60, 34) (3195, 1)
Train on 3195 samples, validate on 798 samples
Epoch 1/15
3195/3195 [==============================] - 0s - loss: 0.6970 - binary_accuracy: 0.4898 - val_loss: 0.6928 - val_binary_accuracy: 0.5276
Epoch 2/15
3195/3195 [==============================] - 0s - loss: 0.6930 - binary_accuracy: 0.5114 - val_loss: 0.6929 - val_binary_accuracy: 0.5025
Epoch 3/15
3195/3195 [==============================] - 0s - loss: 0.6927 - binary_accuracy: 0.5164 - val_loss: 0.6931 - val_binary_accuracy: 0.5025
Epoch 4/15
3195/3195 [======

In [49]:
print(stats)

[[[0.031335573358747926], [0.44710916743939016]], [[0.030854799427534996], [0.41781672502187289]], [[0.022740946245136562], [0.40926178580167749]], [[0.15305565370523522], [0.47199295774758659]], [[0.29723049771566235], [0.56423019434111854]], [[0.36563370169367715], [0.60758494005721786]], [[0.24284237956721089], [0.52159547396710604]], [[0.24469942913792631], [0.50982654898191426]], [[0.23564322397730364], [0.5131946820165838]], [[0.26918824957093657], [0.53728062133379872]], [[0.28206590787106722], [0.53406484553512557]], [[0.1952049762957615], [0.49160434574808548]]]


In [94]:
hist = np.histogram(TCD.y_test)
1-hist[0][-1]/(hist[0][0]+hist[0][-1])

0.50250626566416035

In [43]:
# confusion_matrix(TCD.y_test,TCD.y_test_hat)
# list(zip(TCD.y_test[:100],TCD.y_test_hat[:100]))
np.sum(np.histogram(TCD.y_test_hat)[0])

24913

In [44]:
TCD.model.evaluate(TCD.X_test,TCD.y_test)

24608/24913 [============================>.] - ETA: 0s

[0.52790511442393295, 0.74992975554931163]